# Beam Search (빔 서치) 디코딩

번역 모델이 단어를 내뱉는 방법은 여러 가지가 있습니다. 어떻게 선택하느냐에 따라 번역의 품질이 달라집니다.

## 🔦 비유: 미로 찾기
깜깜한 미로에서 출구를 찾는다고 상상해봅시다.

1.  **Greedy Search (탐욕적 탐색)**: **"근시안적인 등산객"**
    - 갈림길마다 지금 당장 제일 좋아 보이는 길로만 갑니다.
    - 장점: 빠릅니다. 고민 없이 성큼성큼 가니까요.
    - 단점: 지금 당장은 좋아 보였는데 나중에 막다른 길이나 낭떠러지가 나올 수 있습니다. 한 번 선택하면 되돌아오지 않습니다.

2.  **Beam Search (빔 서치)**: **"여러 명의 정찰대 (Scout Team)"**
    - 혼자 가지 않고 **K명(Beam Width)**의 정찰대를 동시에 보냅니다.
    - 각자 자기가 있는 곳에서 갈 수 있는 길을 다 탐색해보고, 그 중에서 **가장 유망한 길 상위 K개**만 남기고 나머지는 버립니다.
    - 이렇게 계속 K개의 최적 경로를 유지하면서 전진합니다.
    - 장점: 혼자 가는 것보다 훨씬 더 좋은 길(정확한 문장)을 찾을 확률이 높습니다.
    - 단점: 계산량이 많아져서 조금 느립니다.

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn

### Beam Search 구현 구현 (핵심 로직)

1.  **초기화**: 시작 토큰(`<sos>`) 하나를 가진 빔(경로) 1개로 시작합니다.
2.  **확장 (Expand)**: 현재 유지하고 있는 모든 빔(경로)에서 다음 단어 후보들을 모두 펼쳐봅니다. (빔 개수 $\times$ 단어장 크기)
3.  **평가 및 가지치기 (Pruning)**: 확장된 수많은 경로 중에서 **점수(확률)가 가장 높은 상위 K개**만 남기고 나머지는 다 자릅니다.
4.  **반복**: 문장이 끝날 때(`<eos>`)까지 2~3번을 반복합니다.

In [2]:
# Beam Search 기본 함수 (이해를 돕기 위한 간소화 버전)
def beam_search(decoder, hidden, context, beam_width=3, max_length=10):
    # 빔 초기화: [ [현재 시퀀스 리스트], 누적 점수, 현재 히든 상태 ]
    # 처음엔 아무것도 없으니 빈 리스트와 점수 1.0(확률 100%)로 시작
    sequences = [[[], 1.0, hidden]]
    
    for _ in range(max_length):      # 최대 길이만큼 반복
        all_candidates = []          # 이번 턴에 확장된 모든 후보들을 담을 곳

        # 현재 살아남은 빔(경로)들을 하나씩 확장
        for seq, score, hidden in sequences:
            # 디코더 입력 준비 (직전 단어가 입력이 됨, 없으면 0)
            last_token = seq[-1] if seq else 0
            decoder_input = torch.tensor([last_token]) 
            
            # 디코더 한 스텝 전진! -> 다음 단어 로짓(output)과 새로운 상태(hidden)
            output, hidden = decoder(decoder_input, hidden, context)
            
            # 확률(Softmax)을 구하고 상위 K(beam_width)개만 뽑음
            probs = F.softmax(output, dim=1)
            top_probs, top_indices = torch.topk(probs, beam_width)

            # 뽑힌 상위 K개 후보를 각각 새로운 경로로 만듦
            for i in range(beam_width):
                token = top_indices[0][i].item()      # 단어 ID
                prob = top_probs[0][i].item()         # 그 단어일 확률
                
                # 새 후보 생성: (경로 추가, 점수 갱신, 상태 보존)
                candidate = [
                    seq + [token],    # 기존 경로에 새 단어 붙임
                    score * prob,     # 확률은 계속 곱해 나감 (P(A,B) = P(A) * P(B))
                    hidden            # 다음 스텝을 위해 상태 보존
                ]
                all_candidates.append(candidate)

        # 모든 후보들 중 점수가 가장 높은 상위 K개만 다시 선발 (가지치기)
        sequences = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width]
    
    # 반복이 끝나면 가장 점수 높은 1등 경로 반환
    return sequences[0][0]

### ⚠️ 개선: 로그 확률 (Log Probability) 사용
확률끼리 계속 곱하면($0.9 \times 0.8 \times 0.5...$) 값이 0에 너무 가까워져서 컴퓨터가 계산하기 힘들어집니다 (Underflow 문제).
그래서 **로그(Log)**를 취해서 **'곱하기'를 '더하기'로 바꿉니다.**
$$ \log(A \times B) = \log(A) + \log(B) $$
이렇게 하면 값이 작아져도 안전하게 계산할 수 있습니다.

In [3]:
# 개선된 Beam Search (<eos> 처리 + Log 확률 사용)
def beam_search_eos(decoder, hidden, context, beam_width=3, max_length=10, sos_id=0, eos_id=2):
    # (시퀀스, 로그 점수 합, 히든 상태)로 초기화
    # 로그 확률이니까 곱셈의 항등원 1(log 1 = 0)인 0.0으로 시작
    sequences = [([sos_id], 0.0, hidden)]

    for _ in range(max_length):
        all_candidates = []
        
        # 현재 빔 확장
        for seq, score, h in sequences:
            # 이미 끝난 문장(<eos>)이면 더 이상 확장하지 않고 그대로 후보에 넣음
            if seq[-1] == eos_id:
                all_candidates.append((seq, score, h))
                continue

            decoder_input = torch.tensor([seq[-1]])
            output, h2 = decoder(decoder_input, h, context)

            # Log Softmax 사용
            log_probs = F.log_softmax(output, dim=1)
            top_logp, top_idx = torch.topk(log_probs, beam_width)

            for i in range(beam_width):
                tok = top_idx[0, i].item()
                log_p = top_logp[0, i].item()
                
                cand = (
                    seq + [tok],      # 시퀀스 확장
                    score + log_p,    # 점수는 더하기(누적)
                    h2                # 다음 히든 상태
                )
                all_candidates.append(cand)

        # 상위 K개 선발
        sequences = sorted(all_candidates, key=lambda x: x[1], reverse=True)[:beam_width]
        
        # 만약 살아남은 모든 빔이 <eos>로 끝났다면 조기 종료
        if all(seq[-1] == eos_id for seq, _, _ in sequences):
            break
    
    best_seq = sequences[0][0]
    return best_seq

In [4]:
# 비교용: Greedy Decode (매 순간 1등만 선택)
def greedy_decode(decoder, hidden, context, max_length=10, sos_id=0, eos_id=2):
    seq = [sos_id]
    h = hidden

    for _ in range(max_length):
        decoder_input = torch.tensor([seq[-1]])
        output, h = decoder(decoder_input, h, context)

        log_probs = F.log_softmax(output, dim=1)
        # 고민 없이 가장 높은 것 1개 선택 (argmax)
        next_tok = torch.argmax(log_probs, dim=1).item()

        seq.append(next_tok)
        if next_tok == eos_id:
            break
    
    return seq

## 테스트를 위한 가짜 디코더 준비
위 함수들이 잘 작동하는지 확인하기 위해 간단한 LSTM 디코더를 만들어봅니다.

In [5]:
class InputFeedingDecoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(InputFeedingDecoder, self).__init__()
        self.embedding = nn.Embedding(output_size, input_size)
        self.lstm = nn.LSTM(input_size + hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden, context):
        embedded = self.embedding(input).unsqueeze(0)
        # 입력에 이전 맥락(Context)을 같이 붙여서 넣어줌
        lstm_input = torch.cat((embedded, context.unsqueeze(0)), dim=2)
        
        output, hidden = self.lstm(lstm_input, hidden)
        output = self.fc(output.squeeze(0))
        return output, hidden

In [6]:
# 단어 맵핑 (숫자 -> 단어)
id2word = {0:"<sos>", 1:"I", 2:"<eos>", 3:"like", 4:"bus", 5:".", 15:"apple", 18:"love", 27:"cat"}

def decode_ids(ids):
    return " ".join(id2word.get(i, "<unk>") for i in ids)

# 테스트 환경 설정
decoder = InputFeedingDecoder(input_size=10, hidden_size=20, output_size=30)
hidden = (torch.zeros(1, 1, 20), torch.zeros(1, 1, 20))
context = torch.zeros(1, 20)

# 실행
greedy_result = greedy_decode(decoder, hidden, context)
beam_result = beam_search_eos(decoder, hidden, context, beam_width=3)

print(f"Greedy 결과: {greedy_result}\n -> {decode_ids(greedy_result)}")
print(f"Beam   결과: {beam_result}\n -> {decode_ids(beam_result)}")

### 결론
- **Greedy**: 빠르지만, 가끔 엉뚱한 문장을 만들 수 있습니다.
- **Beam**: 여러 가능성을 열어두고 끝까지 고민하기 때문에, **더 자연스럽고 정확한 번역문**을 만들 확률이 높습니다. 실제 번역기 서비스는 대부분 Beam Search를 사용합니다.